Deving modification locator script.

evidence.txt is huge.

In [1]:
wc -l evidence.txt

4700965 evidence.txt


Only using 2 columns. The modified sequence and the leading razor protein.

In [2]:
awk -F"\t" '{print $4,$15}' evidence.txt | head

Modified sequence Leading razor protein
_AAAAAAAAAAAAAAAGAGAGAK_ tr|Q53ZR1|Q53ZR1_HUMAN
_AAAAAAAAAAAAAAAGAGAGAK_ tr|Q53ZR1|Q53ZR1_HUMAN
_AAAAAAAAAAAAAAAGAGAGAK_ tr|Q53ZR1|Q53ZR1_HUMAN
_AAAAAAAAAAAAAAAGAGAGAK_ tr|Q53ZR1|Q53ZR1_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN


Assuming the queried amino acid is Q

In [3]:
awk -F"\t" '($4 ~ /Q/){print $4,$15}' evidence_head.txt 

_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN
_AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPAEVGPGAVGER_ sp|O60341|KDM1A_HUMAN


Let's find a bunch with some modifications and a Q and R in it. (uniq doesn't like being cut off so 2>/dev/null it)

In [4]:
awk -F"\t" '($4 ~ /\).*Q.*R.*\)/){print $4,$15}' evidence.txt | uniq 2>/dev/null | head 

_(ac)AAAAVQGGRS(ph)GGSGGCSGAGGASNCGTGSGR_ tr|E9PL01|E9PL01_HUMAN
_(ac)AAAETQSLREQPEM(ox)EDANS(ph)EK_ tr|A0A024QZY5|A0A024QZY5_HUMAN
_(ac)AAAETQSLREQPEMEDANS(ph)EK_ tr|A0A024QZY5|A0A024QZY5_HUMAN
_(ac)AAAETQS(ph)LREQPEMEDANS(ph)EK_ tr|A0A024QZY5|A0A024QZY5_HUMAN
_(ac)AAAET(ph)QSLREQPEMEDANS(ph)EK_ tr|A0A024QZY5|A0A024QZY5_HUMAN
_(ac)AAAETQS(ph)LREQPEMEDANS(ph)EK_ tr|A0A024QZY5|A0A024QZY5_HUMAN
_(ac)AAAPQAPGRGS(ph)LR_ sp|P78345|RPP38_HUMAN
_(ac)AAAPQAPGRGS(ph)LRK_ sp|P78345|RPP38_HUMAN
_(ac)AAARPEAQSRS(ph)SPTPESR_ sp|Q8NCA9|ZN784_HUMAN
_(ac)AAARPEAQSRSS(ph)PTPESR_ sp|Q8NCA9|ZN784_HUMAN


In this list, T and M are modified once so let's use them as our amino acids.

We're looking to find the position of these sequences in a protein. We could use something as simple as substr, but there is a program PeptideMatchCMD_1.0.jar that does this already, so... Let's create an index from our fasta file.

In [6]:
java -jar PeptideMatchCMD_1.0.jar -a index -d uniprot-organism+homo+sapiens.fasta -i uniprot-organism+homo+sapiens

Command line options: -a index -d uniprot-organism+homo+sapiens.fasta -i uniprot-organism+homo+sapiens 
Indexing to directory "/Users/jakewendt/github/unreno/syryu/modification_locator/uniprot-organism+homo+sapiens" ...
Indexing "uniprot-organism+homo+sapiens.fasta" ...
Indexing "uniprot-organism+homo+sapiens.fasta" finished
Time used: 00 hours, 00 mins, 25.542 seconds


In [7]:
java -jar PeptideMatchCMD_1.0.jar -a query -i uniprot-organism+homo+sapiens -q "AAAAVQGGRSGGSGGCSGAGGASNCGTGSGR" -o out

Command line options: -a query -i uniprot-organism+homo+sapiens -q AAAAVQGGRSGGSGGCSGAGGASNCGTGSGR -o out 
Quering...

AAAAVQGGRSGGSGGCSGAGGASNCGTGSGR	has 5 matches

Query is finished.
The result is saved in "out".
Time used: 00 hours, 00 mins, 00.186 seconds


In [8]:
cat out

#Command line options: -a query -i uniprot-organism+homo+sapiens -q AAAAVQGGRSGGSGGCSGAGGASNCGTGSGR -o out 
##Query	Subject	SubjectLength	MatchStart	MatchEnd
AAAAVQGGRSGGSGGCSGAGGASNCGTGSGR	tr|E9PL01|E9PL01_HUMAN	157	2	32	
AAAAVQGGRSGGSGGCSGAGGASNCGTGSGR	sp|Q15005|SPCS2_HUMAN	226	2	32	
AAAAVQGGRSGGSGGCSGAGGASNCGTGSGR	tr|B4DDB9|B4DDB9_HUMAN	226	2	32	
AAAAVQGGRSGGSGGCSGAGGASNCGTGSGR	tr|A0A087WUC6|A0A087WUC6_HUMAN	227	2	32	
AAAAVQGGRSGGSGGCSGAGGASNCGTGSGR	tr|E9PI68|E9PI68_HUMAN	257	2	32	


Five matches, one of which is the expected subject. Also, this program doesn't like the underscores or the modifications so I'll need to remove all of them and then need to find the positions of the modifications myself.

So this gets dirty. Echo the sequence, basically print each character on a separate line, then grep for the desired amino acids, with the line number to get the positions. Add the known start of the sequence to the position of the amino acid and join them together with semicolons.

In [15]:
echo "AAAAVQGGRSGGSGGCSGAGGASNCGTGSGR" | grep -o . | grep -n "[QT]" | awk -F: -v s=157 '{printf(""$1+s";")}' 

163;184;

Even dirtier now. Echo the full raw sequence, strip off the underscores and then match the open parentheses, remember the position, remove the pair of parentheses with contents expected to be 2-4 characters while there are open parentheses, then join the positions together with semicolons.

In [16]:
echo "_(ac)AAAAVQGGRS(ph)GGSGGCSGAGGASNCGTGSGR_" | sed 's/_//g' | awk -v s=157 '{ while(( m = match($0,/\(/) ) > 0 ){ printf(s+m-1";"); sub(/\(.{2,4}\)/,"",$0); } }'

157;167;

Really in it now. Find the intersection of modified amino acids and desired amino acids. Gonna use comm here. Its meant to compare 2 sorted lists and report.

In [18]:
comm -12 <( echo "163;184" | sed 's/;/\n/g' | sort ) <( echo "157;167" | sed 's/;/\n/g' | sort )

Of course, in this case the intersection is empty. However, to see this in action...

In [14]:
comm -12 <( echo "1;10;50;157;184" | sed 's/;/\n/g' | sort ) <( echo "157;167;184;200" | sed 's/;/\n/g' | sort ) | tr "\n" ";"

157;184;

We can remove those trailing semicolons if the string is assigned to a variable.

In [21]:
x="157;184;"
echo $x
echo ${x%?}

157;184;
157;184
